# Autofocus


In [1]:
%load_ext autoreload
%autoreload 2

In [80]:
import os
import shutil
from pathlib import Path

import aps
import aps.ai.autoalignment.beamline28IDB.optimization.common as opt_common
import aps.ai.autoalignment.beamline28IDB.optimization.configs as configs
import aps.ai.autoalignment.beamline28IDB.optimization.movers as movers
import joblib
import matplotlib.pyplot as plt
import numpy as np
import optuna
from aps.ai.autoalignment.beamline28IDB.facade.focusing_optics_factory import (
    ExecutionMode,
    focusing_optics_factory_method,
)
from aps.ai.autoalignment.beamline28IDB.optimization.optuna_botorch import OptunaOptimizer
from aps.ai.autoalignment.beamline28IDB.simulation.facade.focusing_optics_interface import (
    Layout,
    get_default_input_features,
)
from aps.ai.autoalignment.common.simulation.facade.parameters import Implementors
from aps.ai.autoalignment.common.util import clean_up
from aps.ai.autoalignment.common.util.common import AspectRatio, ColorMap, PlotMode
from aps.ai.autoalignment.common.util.shadow.common import (
    EmptyBeamException,
    HybridFailureException,
    PreProcessorFiles,
    load_shadow_beam,
)
from aps.ai.autoalignment.common.util.wrappers import get_distribution_info, load_beam, plot_distribution
from datetime import datetime
import joblib

%matplotlib inline

In [3]:
def select_best_trial_params(trials):
    rms_metrics = []
    for t in trials:
        vals = t.values
        rms_metrics.append(np.sum(np.array(vals) ** 2) ** 0.5)
    idx = np.argmin(rms_metrics)
    return trials[idx].params, trials[idx].values

In [4]:
root_dir = Path(aps.__path__[0]).parents[0]
work_dir = root_dir / "work_directory/28-ID"
os.chdir(work_dir)

In [5]:
input_beam_path = "primary_optics_system_beam.dat"

In [6]:
plot_mode = PlotMode.INTERNAL
aspect_ratio = AspectRatio.AUTO
color_map = ColorMap.VIRIDIS
DEFAULT_RANDOM_SEED = np.random.randint(100000)
print(DEFAULT_RANDOM_SEED)

60650


In [7]:
move_motors_ranges = {
    "hb_1": [-200, 200],  # in V
    "hb_2": [-200, 200],  # in V
    "hb_pitch": [-0.02, 0.02],  # in mrad
    "hb_trans": [-50, 50],  # in microns
    "vb_bender": [-800, 900],  # in volt
    "vb_pitch": [-0.02, 0.02],  # in mrad
    "vb_trans": [-50, 50],  # in microns
}

In [29]:
print("Simulation parameters:")
print(exp_params)

Simulation parameters are:
{'xrange': [-0.7020000000000001, 0.7020000000000001], 'yrange': [-0.8320000000000001, 0.8320000000000001], 'nbins_h': 1024, 'nbins_v': 1024, 'do_gaussian_fit': False, 'implementor': 0, 'random_seed': 60650}


In [30]:
print("motors and movement ranges:")
print(move_motors_ranges)

motors and movement ranges:
{'hb_1': [-200, 200], 'hb_2': [-200, 200], 'hb_pitch': [-0.02, 0.02], 'hb_trans': [-50, 50], 'vb_bender': [-800, 900], 'vb_pitch': [-0.02, 0.02], 'vb_trans': [-50, 50]}


In [10]:
input_features = get_default_input_features(layout=Layout.AUTO_FOCUSING)
focusing_system = opt_common.reinitialize(
    input_beam_path=input_beam_path, input_features=input_features, layout=Layout.AUTO_FOCUSING
)

In [11]:
clean_up()
input_beam = load_shadow_beam(input_beam_path)

focusing_system = focusing_optics_factory_method(
    execution_mode=ExecutionMode.SIMULATION,
    implementor=Implementors.SHADOW,
    bender=True,
)

focusing_system.initialize(
    input_photon_beam=input_beam,
    rewrite_preprocessor_files=PreProcessorFiles.NO,
    layout=Layout.AUTO_FOCUSING,
    input_features=input_features,
)

In [12]:
beam, hist, dw = opt_common.get_beam_hist_dw(focusing_system=focusing_system, photon_beam=None, **exp_params)

write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Upstream_HKB_bender_profile.dat written to disk.
write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Downstream_HKB_bender_profile.dat written to disk.


In [13]:
centroid_ground, *_ = opt_common.get_centroid_distance(photon_beam=beam, **exp_params)
sigma_ground, *_ = opt_common.get_sigma(photon_beam=beam, **exp_params)

In [20]:
print(f"centroid_ground: {centroid_ground:4.3e}, sigma_ground: {sigma_ground:4.3e}")

centroid_ground: 4.907e-04, sigma_ground: 4.280e-03


In [21]:
mots = list(move_motors_ranges.keys())
initial_absolute_positions = {k: movers.get_absolute_positions(focusing_system, k)[0] for k in mots}
print(initial_absolute_positions)

{'hb_1': -168.0, 'hb_2': -161.0, 'hb_pitch': 3.0000000000000506, 'hb_trans': 0.0, 'vb_bender': 419.0, 'vb_pitch': 3.0000000000000506, 'vb_trans': 0.0}


In [22]:
initial_movement, focusing_system, (beam, hist, dw) = opt_common.get_random_init(
    focusing_system, motor_types_and_ranges=move_motors_ranges, intensity_sum_threshold=6.5e3,**exp_params
)

write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Upstream_HKB_bender_profile.dat written to disk.
Random guess [-118.7268670157171, -191.66760073697037, -0.010932246946648379, -35.348001003038796, -518.7483371401597, 0.015831399183874394, -32.765187255199535] produces beam out of bounds. Trying another guess.
write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Upstream_HKB_bender_profile.dat written to disk.
write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Downstream_HKB_bender_profile.dat written to disk.
write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Upstream_HKB_bender_profile.dat written to disk.
Random guess [9.289885379647245, -198.97268554447, 0.013300261381974345, -23.369529737319063, 756.687421230924, -0.014122828384928877, -20.89126627749153] produces b

### We can consider this the actual start of the experiment

In [56]:
focusing_system = movers.move_motors(
    focusing_system,
    list(move_motors_ranges.keys()),
    initial_movement + np.array(list(initial_absolute_positions.values())),
    movement="absolute",
)

centroid_init, beam_init, hist_init, dw_init = opt_common.get_centroid_distance(
    focusing_system=focusing_system, **exp_params
)
sigma_init = opt_common._get_sigma_from_dw(dw_init)
print(centroid_init, sigma_init)

0.02469957678409063 0.12173953326723234


In [57]:
optimization_params = {
    "sum_intensity_soft_constraint": 7e3,
    "sum_intensity_hard_constraint": 6.5e3,
    "centroid_sigma_hard_thresholds_tuple": [0.025, 0.12],
    "n_pitch_trans_motor_trials": 50,
    "n_all_motor_trials": 50,
}

In [58]:
opt_trial = OptunaOptimizer(
    focusing_system,
    motor_types=list(move_motors_ranges.keys()),
    loss_parameters=["centroid", "sigma"],
    multi_objective_optimization=True,
    **exp_params
)

In [59]:
constraints = {"sum_intensity": optimization_params["sum_intensity_soft_constraint"]}
moo_thresholds = None
if optimization_params["centroid_sigma_hard_thresholds_tuple"] is not None:
    moo_thresholds = {
        "centroid": optimization_params["centroid_sigma_hard_thresholds_tuple"][0],
        "sigma": optimization_params["centroid_sigma_hard_thresholds_tuple"][1],
    }

opt_trial.set_optimizer_options(
    motor_ranges=list(move_motors_ranges.values()),
    raise_prune_exception=True,
    use_discrete_space=True,
    sum_intensity_threshold=optimization_params["sum_intensity_hard_constraint"],
    constraints=constraints,
    moo_thresholds=moo_thresholds
)

[I 2022-11-15 01:49:41,287] A new study created in memory with name: no-name-a23343f0-851a-46f8-9df2-1b3526bb3503


In [60]:
print("Optimizing only the pitch and translation motors")

Optimizing only the pitch and translation motors


In [61]:
opt_trial.trials(
    optimization_params["n_pitch_trans_motor_trials"], trial_motor_types=["hb_pitch", "hb_trans", "vb_pitch", "vb_trans"]
)

/Users/skandel/code/oasys/AI-ML_Control_System/aps/ai/autoalignment/beamline28IDB/optimization/optuna_botorch.py:285: ExperimentalWarning: PartialFixedSampler is experimental (supported from v2.4.0). The interface can change in the future.
  partial_sampler = optuna.samplers.PartialFixedSampler(fixed_params, self._base_sampler)
/Applications/Oasys1.3.app/Contents/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/optuna/trial/_trial.py:618: UserWarning: Fixed parameter 'hb_pitch' with value 0.0 is out of range for distribution FloatDistribution(high=0.01995, log=False, low=-0.02, step=0.00017).
  warnings.warn(
/Applications/Oasys1.3.app/Contents/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/optuna/trial/_trial.py:618: UserWarning: Fixed parameter 'vb_pitch' with value 0.0 is out of range for distribution FloatDistribution(high=0.0192, log=False, low=-0.02, step=0.0014).
  warnings.warn(
[I 2022-11-15 01:49:45,081] Trial 0 finished with values: 

Pruning trial with parameters [0.0, 0.0, -0.00793, -39.0, 0.0, -0.018600000000000002, 35.5]


[I 2022-11-15 01:49:45,459] Trial 3 finished with values: [0.09288144023162598, 0.12994353474509454] and parameters: {'hb_1': 0.0, 'hb_2': 0.0, 'hb_pitch': 0.0068600000000000015, 'hb_trans': -20.0, 'vb_bender': 0.0, 'vb_pitch': 0.0023999999999999994, 'vb_trans': -45.5}. 
[I 2022-11-15 01:49:45,583] Trial 4 pruned. 
[I 2022-11-15 01:49:45,707] Trial 5 finished with values: [0.09966507800956736, 0.11107652324533417] and parameters: {'hb_1': 0.0, 'hb_2': 0.0, 'hb_pitch': 0.016380000000000002, 'hb_trans': 46.0, 'vb_bender': 0.0, 'vb_pitch': 0.0192, 'vb_trans': 11.5}. 


Pruning trial with parameters [0.0, 0.0, 0.007710000000000002, -9.0, 0.0, -0.0074, 44.0]


[I 2022-11-15 01:49:45,834] Trial 6 pruned. 
[I 2022-11-15 01:49:45,958] Trial 7 finished with values: [0.07124993682187172, 0.11325412800769877] and parameters: {'hb_1': 0.0, 'hb_2': 0.0, 'hb_pitch': -0.01082, 'hb_trans': 27.0, 'vb_bender': 0.0, 'vb_pitch': 0.0052, 'vb_trans': 15.0}. 


...

[I 2022-11-15 01:49:46,085] Trial 8 finished with values: [0.08129030284568883, 0.1225594948280762] and parameters: {'hb_1': 0.0, 'hb_2': 0.0, 'hb_pitch': -0.01592, 'hb_trans': -1.0, 'vb_bender': 0.0, 'vb_pitch': 0.0023999999999999994, 'vb_trans': -17.5}. 
[I 2022-11-15 01:49:46,209] Trial 9 finished with values: [0.09139496756133347, 0.13094778497563941] and parameters: {'hb_1': 0.0, 'hb_2': 0.0, 'hb_pitch': -0.01507, 'hb_trans': -23.5, 'vb_bender': 0.0, 'vb_pitch': 0.0178, 'vb_trans': -6.0}. 
[I 2022-11-15 01:49:46,335] Trial 10 finished with values: [0.01596606645501266, 0.1256059497707211] and parameters: {'hb_1': 0.0, 'hb_2': 0.0, 'hb_pitch': -0.002489999999999999, 'hb_trans': -10.0, 'vb_bender': 0.0, 'vb_pitch': -0.011600000000000001, 'vb_trans': -44.5}. 
[I 2022-11-15 01:49:46,463] Trial 11 finished with values: [0.02943155892808979, 0.12933276620403555] and parameters: {'hb_1': 0.0, 'hb_2': 0.0, 'hb_pitch': -0.002149999999999999, 'hb_trans': -23.5, 'vb_bender': 0.0, 'vb_pitch':

Pruning trial with parameters [0.0, 0.0, -0.01133, 40.0, 0.0, 0.014999999999999996, 49.5]


[I 2022-11-15 01:51:02,428] Trial 46 finished with values: [0.03257030308664664, 0.11187108346940587] and parameters: {'hb_1': 0.0, 'hb_2': 0.0, 'hb_pitch': 0.008050000000000002, 'hb_trans': 47.5, 'vb_bender': 0.0, 'vb_pitch': -0.0074, 'vb_trans': -46.5}. 
[I 2022-11-15 01:51:04,518] Trial 47 finished with values: [0.015634889859109883, 0.1090215464183636] and parameters: {'hb_1': 0.0, 'hb_2': 0.0, 'hb_pitch': 0.004650000000000001, 'hb_trans': 47.5, 'vb_bender': 0.0, 'vb_pitch': 0.009399999999999999, 'vb_trans': 30.5}. 
[I 2022-11-15 01:51:06,644] Trial 48 pruned. 


Pruning trial with parameters [0.0, 0.0, 0.0058400000000000014, 49.5, 0.0, 0.0192, 50.0]


[I 2022-11-15 01:51:08,804] Trial 49 finished with values: [0.09982961415813947, 0.11180803738847712] and parameters: {'hb_1': 0.0, 'hb_2': 0.0, 'hb_pitch': 0.018930000000000006, 'hb_trans': 50.0, 'vb_bender': 0.0, 'vb_pitch': -0.00040000000000000105, 'vb_trans': 6.0}. 


In [84]:
datetime_str = datetime.strftime(datetime.now(), "%Y:%m:%d:%H:%M")
joblib.dump(opt_trial.study.trials, f"optimization_checkpoint_50_{datetime_str}.pkl")

['optimization_checkpoint_50_2022:11:15:02:03.pkl']

In [62]:
print("Optimizing all motors together")

Optimizing all motors together


In [63]:
opt_trial.trials(optimization_params["n_all_motor_trials"])

[I 2022-11-15 01:51:20,134] Trial 50 pruned. 


write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Upstream_HKB_bender_profile.dat written to disk.
Pruning trial with parameters [21.0, -115.0, 0.0027800000000000012, 40.0, 124.0, -0.006, -41.5]
write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Upstream_HKB_bender_profile.dat written to disk.
write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Downstream_HKB_bender_profile.dat written to disk.


[I 2022-11-15 01:51:22,509] Trial 51 finished with values: [0.02830217284768545, 0.16470550537825013] and parameters: {'hb_1': 73.0, 'hb_2': 39.0, 'hb_pitch': -0.001299999999999999, 'hb_trans': 40.5, 'vb_bender': -160.0, 'vb_pitch': -0.0102, 'vb_trans': -36.5}. 


write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Upstream_HKB_bender_profile.dat written to disk.
write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Downstream_HKB_bender_profile.dat written to disk.


[I 2022-11-15 01:51:25,180] Trial 52 finished with values: [0.06358200910227427, 0.3231781529717705] and parameters: {'hb_1': -3.0, 'hb_2': 34.0, 'hb_pitch': -0.0036799999999999992, 'hb_trans': 19.0, 'vb_bender': -478.0, 'vb_pitch': -0.006, 'vb_trans': -35.0}. 


...

write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Upstream_HKB_bender_profile.dat written to disk.
Pruning trial with parameters [152.0, -110.0, -0.001129999999999999, 11.0, 495.0, -0.018600000000000002, -15.0]
write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Upstream_HKB_bender_profile.dat written to disk.
write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Downstream_HKB_bender_profile.dat written to disk.


[I 2022-11-15 01:58:38,650] Trial 98 finished with values: [0.0168775024881796, 0.055820064742191984] and parameters: {'hb_1': -115.0, 'hb_2': 3.0, 'hb_pitch': 0.012300000000000002, 'hb_trans': 0.0, 'vb_bender': -38.0, 'vb_pitch': -0.0088, 'vb_trans': -35.0}. 


write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Upstream_HKB_bender_profile.dat written to disk.
write_shadow_surface: File for SHADOW /Users/skandel/code/oasys/AI-ML_Control_System/work_directory/28-ID/Downstream_HKB_bender_profile.dat written to disk.


/Applications/Oasys1.3.app/Contents/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/oasys/util/oasys_util.py:283: RuntimeWarning: invalid value encountered in true_divide
  frequency = histogram/numpy.sum(histogram)
/Applications/Oasys1.3.app/Contents/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/oasys/util/oasys_util.py:292: RuntimeWarning: invalid value encountered in true_divide
  frequency = histogram/numpy.sum(histogram)
[I 2022-11-15 01:58:41,598] Trial 99 pruned. 


Pruning trial with parameters [109.0, 135.0, 0.008730000000000002, 34.0, 738.0, -0.011600000000000001, -42.5]


In [74]:
optimal_params, values = select_best_trial_params(opt_trial.study.best_trials)

In [75]:
optimal_params, values

({'hb_1': -87.0,
  'hb_2': -29.0,
  'hb_pitch': -0.0033399999999999992,
  'hb_trans': -37.5,
  'vb_bender': -88.0,
  'vb_pitch': -0.0017999999999999995,
  'vb_trans': -15.0},
 [0.011216053810391211, 0.05278501466031434])

In [76]:
initial_movement

[135.16917904768525,
 -36.05448803397144,
 -0.014481142591426997,
 41.85642548717669,
 90.49112876710194,
 0.0017745091189553322,
 19.31957193850937]

In [77]:
opt_trial._loss_fn_this(list(optimal_params.values()))
#plot_distribution(beam=opt_trial.beam_state.photon_beam, implementor=Implementors.SHADOW)# **exp_params)

array([0.00926111, 0.05276798])

In [85]:
datetime_str = datetime.strftime(datetime.now(), "%Y:%m:%d:%H:%M")
joblib.dump(opt_trial.study.trials, f"optimization_final_100_{datetime_str}.pkl")

['optimization_final_100_2022:11:15:02:03.pkl']

error sending registration message to CA repeater daemon was "Network is unreachable"
error sending registration message to CA repeater daemon was "Network is unreachable"
error sending registration message to CA repeater daemon was "Network is unreachable"
error sending registration message to CA repeater daemon was "Network is unreachable"
error sending registration message to CA repeater daemon was "Network is unreachable"
error sending registration message to CA repeater daemon was "Network is unreachable"
error sending registration message to CA repeater daemon was "Network is unreachable"
error sending registration message to CA repeater daemon was "Network is unreachable"
error sending registration message to CA repeater daemon was "Network is unreachable"
error sending registration message to CA repeater daemon was "Network is unreachable"
error sending registration message to CA repeater daemon was "Network is unreachable"
error sending registration message to CA repeater daem